In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import sys
import gc

In [2]:
customer = pd.read_csv('input/original/Customer.csv')
customer.columns = ['CustomerIdx', 'Customer_Sector', 'Customer_Subsector', 'Customer_Region', 'Customer_Country']
print(customer.shape)
customer.head()

(3471, 5)


,CustomerIdx,Customer_Sector,Customer_Subsector,Customer_Region,Customer_Country
0,2975,Banks and Intermediaries,Bank,Americas,BARBADOS
1,1594,Asset Managers & Hedge Funds,NaN,Americas,BERMUDA
2,399,Corporation,Corp - Comm. & Prof. Services,Americas,BERMUDA
3,836,Asset Owners,Insurance,Americas,BERMUDA
4,816,Asset Owners,Insurance,Americas,BERMUDA


In [2]:
full = pd.read_csv('input/generated/trade_generated.csv')
print(full.shape)
full.head()

(9516859, 9)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,PredictionIdx,WeekID,Week_of_Month,Year
0,Buy,1856,1.0,9230,2,trade,68,2,2017
1,Buy,1856,1.0,9230,11,trade,114,1,2017
2,Buy,1856,1.0,9230,4,trade,138,1,2018
3,Buy,1856,1.0,9230,4,trade,139,2,2018
4,Buy,1746,1.0,13099,11,trade,52,1,2016


In [3]:
cust_bond_feat = full[['CustomerIdx', 'IsinIdx']].drop_duplicates().reset_index(drop=True)
print(cust_bond_feat.shape)
cust_bond_feat.head()

(242379, 2)


,CustomerIdx,IsinIdx
0,1856,9230
1,1746,13099
2,2398,9131
3,2398,14053
4,2398,8575


In [4]:
full['Prev_WeekID'] = full.WeekID-1

In [5]:
trade = pd.read_csv('input/original/Trade.csv', parse_dates=['TradeDateKey'])
trade.rename(columns = {'TradeDateKey':'DateKey'},inplace = True)
print(trade.shape)
trade.head()

(6762021, 8)


,DateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest
0,2016-12-07,2789,8478,Sell,653168.0,0.0,Unknown,1.0
1,2017-03-29,2574,14562,Buy,1656487.0,0.0,Unknown,1.0
2,2017-04-18,2574,4747,Buy,939673.0,0.0,Unknown,1.0
3,2017-03-10,2574,9885,Sell,708082.0,0.0,Unknown,1.0
4,2016-11-16,2574,8885,Buy,1147709.0,0.0,Unknown,1.0


In [8]:
cust_bond_feat = trade[['CustomerIdx', 'IsinIdx']].drop_duplicates().reset_index(drop=True)
print(cust_bond_feat.shape)
cust_bond_feat.head()

(1395659, 2)


,CustomerIdx,IsinIdx
0,2789,8478
1,2574,14562
2,2574,4747
3,2574,9885
4,2574,8885


In [11]:
import datetime
import calendar
def week_of_month(tgtdate):
    tgtdate = tgtdate.to_pydatetime()

    days_this_month = calendar.mdays[tgtdate.month]
    for i in range(1, days_this_month):
        d = datetime.datetime(tgtdate.year, tgtdate.month, i)
        if d.day - d.weekday() > 0:
            startdate = d
            break
    # now we canuse the modulo 7 appraoch
    return (tgtdate - startdate).days //7 + 1

trade['Year'] = trade['DateKey'].dt.year
trade['Month'] = trade['DateKey'].dt.month
trade['Week_of_Month'] = trade['DateKey'].apply(week_of_month)
trade['TimeHash'] = trade.Year*1000 + trade.Month*10 + trade.Week_of_Month
timehash_weekid = pd.read_csv('input/generated/TimeHash-WeekID.csv', usecols=['TimeHash', 'WeekID'])
trade = trade.merge(timehash_weekid)
trade['Prev_WeekID'] = trade.WeekID-1
trade.head()

,DateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest,Year,Month,Week_of_Month,TimeHash,WeekID,Prev_WeekID
0,2016-12-07,2789,8478,Sell,653168.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56
1,2016-12-06,2398,9452,Sell,716873.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56
2,2016-12-06,2398,12036,Sell,785588.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56
3,2016-12-08,2574,6921,Buy,697156.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56
4,2016-12-07,2574,7762,Sell,513203.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56


Time >= 23rd october 2018 or WeekID >=111 is P6M

In [12]:
trade[(trade.WeekID == 110)].DateKey.max()

Timestamp('2017-10-22 00:00:00')

In [13]:
trade[(trade.CustomerInterest==1) & (trade.WeekID >  110)].head()

,DateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest,Year,Month,Week_of_Month,TimeHash,WeekID,Prev_WeekID
1250953,2017-11-17,1856,13323,Sell,1106531.0,0.0,Unknown,1.0,2017,11,2,2017112,115,114
1250954,2017-11-13,1758,7151,Sell,514293.0,0.0,Unknown,1.0,2017,11,2,2017112,115,114
1250955,2017-11-13,1856,10604,Sell,951442.0,0.0,Unknown,1.0,2017,11,2,2017112,115,114
1250956,2017-11-16,2574,4745,Sell,508755.0,0.0,Unknown,1.0,2017,11,2,2017112,115,114
1250957,2017-11-16,1856,6672,Sell,847925.0,0.0,Unknown,1.0,2017,11,2,2017112,115,114


In [18]:
#Need to change to this bond last week

# No of times a user buys/sells this bond in p6m
temp = trade[(trade.CustomerInterest==1) & (trade.WeekID >  110)].copy()
temp = temp.groupby(['CustomerIdx', 'IsinIdx', 'BuySell']).size().reset_index()
temp.columns = ['CustomerIdx', 'IsinIdx', 'BuySell', 'value']
temp = temp.pivot_table(index=['CustomerIdx', 'IsinIdx'], columns='BuySell', values='value').reset_index().fillna(0)
temp.rename(columns = {'Buy': 'cust_bond_times_buy_p6m', 'Sell': 'cust_bond_times_sell_p6m'}, inplace=True)
temp['cust_bond_times_p6m'] = temp.cust_bond_times_buy_p6m + temp.cust_bond_times_sell_p6m
cust_bond_feat = cust_bond_feat.merge(temp, how='left', on=['CustomerIdx', 'IsinIdx'])
cust_bond_feat.fillna(0, inplace=True)
print(cust_bond_feat.shape)
cust_bond_feat.head()

(1395659, 5)


,CustomerIdx,IsinIdx,cust_bond_times_buy_p6m,cust_bond_times_sell_p6m,cust_bond_times_p6m
0,2789,8478,NaN,NaN,NaN
1,2574,14562,1.0,1.0,2.0
2,2574,4747,NaN,NaN,NaN
3,2574,9885,NaN,NaN,NaN
4,2574,8885,0.0,1.0,1.0


In [19]:
# Has transaction made last week binary

,CustomerIdx,IsinIdx,cust_bond_times_buy_p6m,cust_bond_times_sell_p6m,cust_bond_times_p6m
0,2789,8478,0.0,0.0,0.0
1,2574,14562,1.0,1.0,2.0
2,2574,4747,0.0,0.0,0.0
3,2574,9885,0.0,0.0,0.0
4,2574,8885,0.0,1.0,1.0


In [20]:
trade[trade.TradeStatus != 'Holding'].shape

(2211929, 14)

In [21]:
trade.shape

(6762021, 14)

In [ ]:
#trade['Hash'] = trade.CustomerIdx.astype(str) + trade.IsinIdx.astype(str) + trade.BuySell + trade.WeekID.astype(str)
trade = trade[trade.TradeStatus != 'Holding']
trade.shape

In [ ]:
# Customer Bond Features

In [22]:
# Customer Is bond traded last week
trade.head()

,DateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest,Year,Month,Week_of_Month,TimeHash,WeekID,Prev_WeekID
0,2016-12-07,2789,8478,Sell,653168.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56
1,2016-12-06,2398,9452,Sell,716873.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56
2,2016-12-06,2398,12036,Sell,785588.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56
3,2016-12-08,2574,6921,Buy,697156.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56
4,2016-12-07,2574,7762,Sell,513203.0,0.0,Unknown,1.0,2016,12,1,2016121,57,56


In [ ]:
# Ratio
# Time since last order
# No of transaction since last buying this bond
# Weeks since first order
# Weeks since last order